In [ ]:
import pandas as pd
import yaml
import numpy as np
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
from datetime import datetime

# Função para carregar configuração do YAML
def load_config(config_path):
    with open(config_path, 'r', encoding='utf-8') as file:
        return yaml.safe_load(file)

# Função de pré-processamento do CSV
def preprocess_csv(csv_file):
    df = pd.read_csv(csv_file)
    # Eliminar a primeira coluna identificada pelo rótulo "#"
    if df.columns[0] == '#':
        df = df.drop(columns=['#'])
    # Eliminar as últimas colunas desnecessárias
    columns_to_drop = ["Response Type", "Start Date (UTC)", "Stage Date (UTC)", "Submit Date (UTC)", "Network ID", "Tags"]
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
    # Substituir células vazias por zero
    df = df.fillna(0)
    return df

# Função para calcular as pontuações de cada componente
def calculate_scores(df, components):
    scores = {}
    detailed_scores = {}
    for component, indices in components.items():
        component_scores = df.iloc[:, [i-1 for i in indices if i-1 < df.shape[1]]].apply(pd.to_numeric, errors='coerce').round(0).astype(int)
        scores[component] = int(component_scores.sum(axis=1).values[0])
        detailed_scores[component] = component_scores.values.tolist()[0]
    return scores, detailed_scores

# Função para interpretar as pontuações
def interpret_scores(scores):
    return {component: ('alta' if score >= 20 else 'baixa' if score < 10 else 'moderada') for component, score in scores.items()}

# Função para gerar o gráfico radar e salvar como imagem
def plot_radar_chart(scores):
    labels = list(scores.keys())
    values = list(scores.values())
    values += values[:1]
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    angles += angles[:1]
    
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.fill(angles, values, color='red', alpha=0.25)
    ax.plot(angles, values, color='red', linewidth=2)
    
    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)
    
    plt.title('Pontuações de Inteligência Emocional')
    radar_chart_image_path = 'radar_chart.png'
    plt.savefig(radar_chart_image_path, bbox_inches='tight')
    plt.close(fig)
    return radar_chart_image_path

# Função para gerar o arquivo de saída com as pontuações detalhadas
def generate_output_file(detailed_scores, components, questions):
    output_data = []
    for component, scores in detailed_scores.items():
        for i, score in enumerate(scores):
            question_idx = components[component][i]
            question = questions[question_idx - 1] if question_idx - 1 < len(questions) else "Pergunta não encontrada"
            output_data.append([component, question, score])
    output_df = pd.DataFrame(output_data, columns=['Componente', 'Pergunta', 'Pontuação'])
    output_df.to_csv('EQoutput.csv', index=False)
    print("\nArquivo de saída 'EQoutput.csv' gerado com sucesso.")
    return output_df

# Função para criar o layout do PDF
def create_pdf_layout(output_text, table_data, scores_table, image_path, person_name, evaluation_date):
    elements = []
    styles = getSampleStyleSheet()
    custom_styles = {
        'section': ParagraphStyle(name='Section', fontSize=14, leading=16, spaceAfter=10, fontName='Helvetica-Bold'),
        'subsection': ParagraphStyle(name='Subsection', fontSize=12, leading=14, spaceAfter=6, fontName='Helvetica-Bold')
    }
    
    # Título do documento
    elements.append(Paragraph("Relatório de Pontuações de Inteligência Emocional", styles['Title']))
    elements.append(Spacer(1, 12))
    
    # Adicionar nome e data
    if person_name and evaluation_date:
        elements.append(Paragraph(f"Nome: {person_name}", styles['Normal']))
        elements.append(Paragraph(f"Data da Avaliação: {evaluation_date}", styles['Normal']))
        elements.append(Spacer(1, 12))
    
    # Adicionar seção de pontuações dos componentes
    elements.append(Paragraph("Pontuações dos Componentes", custom_styles['section']))
    elements.append(Spacer(1, 12))
    
    # Adicionar tabela de pontuações dos componentes
    elements.append(Table(scores_table, style=[
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    elements.append(Spacer(1, 12))
    
    # Adicionar pontuação geral de inteligência emocional
    elements.append(Paragraph("Pontuação Geral de Inteligência Emocional:", custom_styles['subsection']))
    elements.append(Paragraph(output_text[0], styles['Normal']))
    elements.append(Spacer(1, 12))
    
    # Adicionar interpretação das pontuações
    elements.append(Paragraph("Interpretação das Pontuações:", custom_styles['subsection']))
    for line in output_text[1:]:
        elements.append(Paragraph(line, styles['Normal']))
        elements.append(Spacer(1, 12))
    
    # Nova página para a tabela de pontuações detalhadas
    elements.append(PageBreak())
    elements.append(Paragraph("Tabela de Pontuações Detalhadas", custom_styles['section']))
    elements.append(Spacer(1, 12))
    
    # Criar tabela de pontuações detalhadas
    table = Table(table_data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    elements.append(table)
    
    # Nova página para o gráfico radar
    elements.append(PageBreak())
    elements.append(Paragraph("Gráfico Radar", custom_styles['section']))
    elements.append(Spacer(1, 12))
    if image_path:
        elements.append(Image(image_path, width=6*inch, height=6*inch))
    
    return elements

# Função para gerar o PDF
def generate_pdf(elements):
    doc = SimpleDocTemplate("EQoutput.pdf", pagesize=letter)
    doc.build(elements)
    print("\nArquivo de saída 'EQoutput.pdf' gerado com sucesso.")

# Função principal para processar o CSV e fornecer a análise
def analyze_emotional_intelligence(csv_file, config_path, person_name, evaluation_date):
    # Carregar configuração
    config = load_config(config_path)
    questions = config['questions']
    components = config['components']
    
    # Pré-processar o arquivo CSV
    df = preprocess_csv(csv_file)
    
    # Calcular pontuações
    scores, detailed_scores = calculate_scores(df, components)
    
    # Fornecer pontuação geral
    overall_score = sum(scores.values())
    
    # Interpretação
    interpretation = interpret_scores(scores)
    
    # Armazenar resultados para o PDF
    output_text = []
    scores_table = [['Componente', 'Pontuação']]
    for component, score in scores.items():
        scores_table.append([component, score])
    
    output_text.append(f"{overall_score}")
    if overall_score >= 100:
        output_text.append("Você é considerado uma pessoa com alta inteligência emocional.")
    elif 50 <= overall_score < 100:
        output_text.append("Você tem uma boa base de inteligência emocional para desenvolver sua capacidade de liderança.")
    else:
        output_text.append("Provavelmente você está abaixo da média em inteligência emocional.")
    
    output_text.append("Interpretação das Pontuações:")
    for component, interp in interpretation.items():
        output_text.append(f"{component}: {interp}")

    # Gerar o gráfico radar e salvar como imagem
    radar_chart_image_path = plot_radar_chart(scores)
    
    # Gerar o arquivo de saída
    output_df = generate_output_file(detailed_scores, components, questions)
    
    # Preparar dados da tabela detalhada
    table_data = [['Componente', 'Pergunta', 'Pontuação']] + output_df.values.tolist()
    
    # Criar layout do PDF
    elements = create_pdf_layout(output_text, table_data, scores_table, radar_chart_image_path, person_name, evaluation_date)
    
    # Gerar o PDF
    generate_pdf(elements)

# Solicitar nome da pessoa avaliada e data da avaliação
person_name = input("Insira o nome da pessoa avaliada: ")
evaluation_date = input("Insira a data da avaliação (dd/mm/yyyy): ")

# Chamar a função para analisar o CSV
csv_file = 'EQresponses.csv'  # Caminho para seu arquivo CSV
config_path = 'EQconfig.yaml'  # Caminho para seu arquivo de configuração
analyze_emotional_intelligence(csv_file, config_path, person_name, evaluation_date)